# CONSIDERACIONES DE USO CON STREAMING

- Streaming Limitado a Modelos de Chat: El streaming sólo se utiliza para los modelos de chat y no para las herramientas.
- Funcionalidad Reducida: Este enfoque limita la funcionalidad del agente ya que no puede invocar herramientas, pero permite probar el streaming con el modelo de chat.

In [20]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

load_dotenv()

True

In [21]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [22]:
model = ChatGroq(temperature=0, model="llama3-70b-8192", streaming=True)

In [23]:
def add_messages(left: list, right: list):
    """Añadir sin sobrescribir."""
    return left + right

In [24]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [25]:
def should_continue(state: AgentState) -> Literal["__end__"]:
    # Siempre terminamos ya que no estamos usando herramientas
    return "__end__"

In [26]:
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


## Definir el Grafo

In [27]:
# Definir un nuevo grafo
workflow = StateGraph(AgentState)


In [28]:
# Definir los dos nodos entre los que ciclará
workflow.add_node("agent", call_model)

In [29]:
# Establecer el punto de entrada como `agent`
workflow.set_entry_point("agent")

In [30]:
# Agregar un borde condicional
workflow.add_conditional_edges("agent", should_continue)

In [31]:
# Finalmente, ¡compilamos!
app = workflow.compile()

In [32]:
mensaje = HumanMessage(content="¿Cuál es el clima en San Francisco? Contesta en español")

In [33]:

inputs = {"messages": [HumanMessage(content="¿Cuál es el clima en San Francisco?")]}
inputs


{'messages': [HumanMessage(content='¿Cuál es el clima en San Francisco?')]}

In [34]:
for output in app.stream(inputs, stream_mode="updates"):
    # stream() genera diccionarios con la salida clave por nombre de nodo
    for key, value in output.items():
        print(f"Salida del nodo '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Salida del nodo 'agent':
---
{'messages': [AIMessage(content='San Francisco, California, tiene un clima mediterráneo con influencia oceánica, caracterizado por veranos frescos y húmedos, e inviernos suaves y lluviosos. A continuación, se presentan las características climáticas de San Francisco:\n\n**Temperatura:**\n\n* Verano (junio a septiembre): la temperatura máxima promedio es de 22°C (72°F), mientras que la temperatura mínima promedio es de 13°C (56°F).\n* Invierno (diciembre a febrero): la temperatura máxima promedio es de 14°C (57°F), mientras que la temperatura mínima promedio es de 8°C (46°F).\n* Primavera (marzo a mayo) y otoño (septiembre a noviembre): las temperaturas son suaves, con máximas promedio de 18°C (64°F) y mínimas promedio de 10°C (50°F).\n\n**Precipitaciones:**\n\n* La mayoría de las precipitaciones ocurren durante los meses de invierno, con un promedio de 20 días de lluvia al mes.\n* La cantidad de precipitaciones anuales es de aproximadamente 600 mm (24 pulga